In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os

In [ ]:
from matplotlib.image import imread

In [ ]:
data_dir = '/kaggle/input/nmlo-contest-4/'
os.listdir(data_dir)

In [ ]:
train_path = data_dir + 'train' + '/train'
test_path = data_dir + 'test' + '/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest',
                               validation_split=0.15, 
                               rescale=1./255)

#we want more images but we too lazy to collect more
#so we can like randomly rotate preexisting imgs from dataset and act like they new imgs

#we can do these weird transformations cuz we're working with fruits which are just glorified blobs
#cant go too crazy with transformations if workin with, like, idk, human faces
#I mean you could still do this but sometimes you'll end up with something that looks very fake

In [ ]:
image_shape=(100,100,3)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,target_size=image_shape[:2],
                                                color_mode='rgb',
                                                batch_size=50,
                                                class_mode='categorical',
                                                subset='training'
                                               )

In [ ]:
val_image_gen = image_gen.flow_from_directory(train_path,target_size=image_shape[:2],
                                                color_mode='rgb',
                                                batch_size=50,
                                                class_mode='categorical',
                                                subset='validation'
                                             )

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout 
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.5))

#OUTPUT LAYER
model.add(Dense(33,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf



In [ ]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',patience=2)

In [ ]:
results = model.fit_generator(train_image_gen,epochs=20,
                              steps_per_epoch=50,
                              validation_data=val_image_gen,
                              callbacks=[early_stop],
                              verbose=1)

In [ ]:
label_map = (train_image_gen.class_indices)
label_map

In [ ]:
from keras.preprocessing import image


# load all images into a list
images = []
for img in sorted(os.listdir(test_path)):
    img = os.path.join(test_path, img)
    img = image.load_img(img, target_size=(100, 100))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
classes

In [ ]:
inv_map = {v: k for k, v in label_map.items()}
inv_map

In [ ]:
C = list((pd.Series(classes)).map(inv_map))
C

In [ ]:
submission = pd.DataFrame({'id':range(5641),'label':C})
submission

In [ ]:
submission.to_csv('FRUITTTTT.csv',index=False)